In [ ]:
# This dataset includes 
# scores from three test scores 
# of students at a (fictional) public school 
# and a variety of personal and socio-economic factors 
# that may have interaction effects upon them.

# https://www.kaggle.com/datasets/desalegngeb/students-exam-scores?resource=download

In [ ]:
# 데이터셋 가져오기

import os
import pandas as pd

path = './data/'
file_orginal = 'Original_data_with_more_rows.csv'
file_expanded = 'Expanded_data_with_more_features.csv'

path_file_original = os.path.join(path,file_orginal)
path_file_expanded = os.path.join(path,file_expanded)

data_original = pd.read_csv(path_file_original)
data_expanded = pd.read_csv(path_file_expanded)

In [79]:
# 데이터셋 살펴보기 - 전체적으로
    # 중복 제거
    # 결측치 규모

# data_original.info() 
# 9 cols 30640 rows
data_expanded.info() 
# 15 cols 30640 rows

data = data_expanded
data.drop_duplicates()

null_count = data.isna().sum()
null_count

null_ratio = data.isna().mean().sort_values(ascending=False)*100
null_ratio

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30641 entries, 0 to 30640
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           30641 non-null  int64  
 1   Gender               30641 non-null  object 
 2   EthnicGroup          28801 non-null  object 
 3   ParentEduc           28796 non-null  object 
 4   LunchType            30641 non-null  object 
 5   TestPrep             28811 non-null  object 
 6   ParentMaritalStatus  29451 non-null  object 
 7   PracticeSport        30010 non-null  object 
 8   IsFirstChild         29737 non-null  object 
 9   NrSiblings           29069 non-null  float64
 10  TransportMeans       27507 non-null  object 
 11  WklyStudyHours       29686 non-null  object 
 12  MathScore            30641 non-null  int64  
 13  ReadingScore         30641 non-null  int64  
 14  WritingScore         30641 non-null  int64  
dtypes: float64(1), int64(4), object(10)


TransportMeans         10.228126
ParentEduc              6.021344
EthnicGroup             6.005026
TestPrep                5.972390
NrSiblings              5.130381
ParentMaritalStatus     3.883685
WklyStudyHours          3.116739
IsFirstChild            2.950295
PracticeSport           2.059332
Unnamed: 0              0.000000
Gender                  0.000000
LunchType               0.000000
MathScore               0.000000
ReadingScore            0.000000
WritingScore            0.000000
dtype: float64

In [105]:
# 데이터셋 살펴보기 - 각 컬럼별로
    # 데이터타입 : 데이터타입 변환이 필요한가
    # 데이터유형 : 어떤 데이터들이 어떻게 있는가
        # 수치형
            # 분포 확인 -> 대표값, 유효범위
            # <- 박스플롯
        # 범주형
            # 분포 확인 : 각 범주별 빈도
            # <- 막대그래프, 파이차트
    # 결측치 규모 : 일괄 삭제할 것인지, 추론할 것인지

data.columns
columns_ordered = [
    'Unnamed: 0'
    , 'ParentMaritalStatus', 'ParentEduc'
    , 'IsFirstChild', 'NrSiblings'
    , 'Gender', 'EthnicGroup'
    , 'TransportMeans', 'LunchType'
    , 'PracticeSport', 'WklyStudyHours', 'TestPrep'
    , 'MathScore', 'ReadingScore', 'WritingScore'
]
data = data.reindex(columns=columns_ordered)
columns = data.columns
columns

column = 'IsFirstChild'
data[column].dtype
data[column].value_counts().sort_values(ascending=False)
# data[column].value_counts(normalize=True).reset_index()
data[column].isna().sum()
data[column].isna().mean()*100

2.9502953558956952

In [ ]:
# 데이터셋 살펴보기 - 컬럼 간 관계성
    # 수치형 
        # data.corr()
        # data.describe()
        # scaler.fit_transform(data)
    # 범주형 
        # 논리적으로
        # 가설 -> 확인 -> 결론

# 결측치 처리
# 이상치 처리

In [113]:
# NrSiblings 형제수
# IsFirstChild 첫째여부

column = 'NrSiblings'
count = data[column].isna().sum()
ratio = data[column].isna().mean()*100
print(f'--{column}\ncount {count}\nratio {ratio}')

column = 'IsFirstChild'
count = data[column].isna().sum()
ratio = data[column].isna().mean()*100
print(f'--{column}\ncount {count}\nratio {ratio}')


# NrSiblings=Null 일때, IsFirstChild 확인

data[data['NrSiblings'].isna()]['IsFirstChild'].isna().sum() 
# null 54
data[data['NrSiblings'].isna()]['IsFirstChild'].value_counts() 
# yes    929
# no     589

# 54 : 현재 형제수 데이터에는 0 이 없음. 즉, 외동에 대해서는 표현이 없음. -> NrSiblings=0.0
# 1518 : 첫째여도 첫째가 아니여도, 어쨌거나 형제가 있다는 뜻. -> NrSiblings=1.0


# IsFirstChild=Null 일때, NrSiblings 확인

data[data['IsFirstChild'].isna()]['NrSiblings'].isna().sum()
# 54
data[data['IsFirstChild'].isna()]['NrSiblings'].value_counts()
# 1.0    245
# 2.0    187
# 0.0    151
# 3.0    147
# 4.0     74
# 5.0     35
# 7.0      6
# 6.0      5

# 54 : 형제수가 0, 다시말해 외동인 경우, 첫째이기도 한 것으로 표현. -> IsFirstChild=yes
# 850 -> ?

--NrSiblings
count 1572
ratio 5.130380862243399
--IsFirstChild
count 904
ratio 2.9502953558956952


yes    929
no     589
Name: IsFirstChild, dtype: int64

In [ ]:
# x, y 분리
    # 수치형 -> 스케일링
    # 범주형 -> 인코딩 - 라벨 or 원핫

# train, test 데이터 분할

# 모델 생성 <- 적절한 모델로 선택
# 모델 학습
# 모델 평가

# 하이퍼파라미터 튜닝